In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf


env = "local"

In [2]:
if env == "local":
    os.chdir("/Users/samuel/Documents/GitHub/QB-GPT/")
else:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/MyDrive/NFL_Challenge/NFL-GPT/NFL data")

In [3]:
os.listdir()

['data_models',
 '.DS_Store',
 'LICENSE',
 'models',
 'README.md',
 '.gitignore',
 '.gitattributes',
 'data_preprocessing',
 'index',
 '.git',
 'notebooks']

In [21]:
training_data = tf.data.Dataset.load("data_models/QBGPT/train_tokens_NFL_GPT")
testing_data = tf.data.Dataset.load("data_models/QBGPT/test_tokens_NFL_GPT")

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 9214814344728754403
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 13130669831927790889


In [22]:
train_length = [i for i,_ in enumerate(training_data)][-1] + 1
test_length = [i for i,_ in enumerate(testing_data)][-1] + 1

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 9214814344728754403
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 13130669831927790889


In [23]:
print("Train length is : ", str(train_length))
print("Test length is : ", str(test_length))

Train length is :  205851
Test length is :  51463


In [24]:
batch_size = 32

training_data = training_data.shuffle(train_length).batch(batch_size)
testing_data = testing_data.shuffle(test_length).batch(batch_size)

## Model classes

In [8]:
def insert_weights(df, w):
  df["weights"] = [w for i in range(df.shape[0])]
  return df

class_weights = pd.read_parquet("models/modeling/QBGPT/class_weights.parquet")

step_range = [(0, 10), (10, 100), (100, 1000), (1000, 10000), (10000, 50000), (50000, 100000), (100000, 300000), (300000, 500000), (500000, 1000000), (1000000, 10000000)]

In [9]:
from collections import Counter
weights = dict(Counter(class_weights["Zone_ID"].to_numpy()))
weights_df = pd.DataFrame(np.array([[k, v] for k,v in weights.items()]), columns = ["Class", "Count"])

weights_dict = {i : weights_df[(weights_df['Count'] > step_range[i][0]) & (weights_df['Count'] <= step_range[i][1])].reset_index(drop = True) for i in range(len(step_range))}
w_dict = {0 : 1,
          1 : 0.9,
          2 : 0.8,
          3 : 0.7,
          4 : 0.6,
          5 : 0.5,
          6 : 0.4,
          7 : 0.3,
          8 : 0.2,
          9 : 0.05,}

weights_dict = {k:insert_weights(v, w_dict[k]) for k,v in weights_dict.items()}

weights_df = pd.concat(list(weights_dict.values())).reset_index(drop = True)

In [10]:
weights_inv = {v[0] : v[2] for v in weights_df.values}

In [11]:
def scheduler(epoch, lr):
  if epoch < 1:
    return 3e-3
  elif (epoch >= 1) & (epoch < 2):
    return 2e-3
  elif (epoch >= 2) & (epoch < 3):
    return 1e-3
  elif (epoch >= 3) & (epoch < 5):
    return 5e-4
  elif (epoch >= 5) & (epoch < 7):
    return 1e-4
  else:
    return 5e-5


schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [29]:
from models.modeling.QBGPT.models import QBGPT, LargeQBGPT, XLargeQBGPT
from models.modeling.QBGPT.losses_and_metrics import CustomSparseCategoricalAccuracy, CustomTopKAccuracy, CustomSparseCategoricalCrossentropy

moves_to_pred = 11164
input_size = 11166
starts_size = 1985
scrimmage_size = 100
positions_id = 29
temp_ids = 52

model_large = QBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = 2,
                    type_vocab_size = 3,
                    playtype_vocab_size = 9,
                    embedding_dim = 512,
                    hidden_dim = 512,
                    to_pred_size = moves_to_pred)

model_medium = QBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = 2,
                    type_vocab_size = 3,
                    playtype_vocab_size = 9,
                    embedding_dim = 256,
                    hidden_dim = 256,
                    to_pred_size = moves_to_pred)

model_small = QBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = 2,
                    type_vocab_size = 3,
                    playtype_vocab_size = 9,
                    embedding_dim = 128,
                    hidden_dim = 128,
                    to_pred_size = moves_to_pred)

large_model = LargeQBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = 2,
                    type_vocab_size = 3,
                    playtype_vocab_size = 9,
                    embedding_dim = 256,
                    hidden_dim = 256,
                    to_pred_size = moves_to_pred)

xlarge_model = XLargeQBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = 2,
                    type_vocab_size = 3,
                    playtype_vocab_size = 9,
                    embedding_dim = 256,
                    hidden_dim = 256,
                    to_pred_size = moves_to_pred)

In [26]:
essai = list(training_data)[0][0]

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 9214814344728754403


In [27]:
essai

{'OffDef': <tf.Tensor: shape=(32, 256), dtype=int64, numpy=
 array([[1, 1, 1, ..., 1, 1, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [0, 0, 0, ..., 0, 0, 0]])>,
 'attention_mask': <tf.Tensor: shape=(32, 256), dtype=int64, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])>,
 'position_ids': <tf.Tensor: shape=(32, 256), dtype=int64, numpy=
 array([[27, 27, 27, ..., 28, 28, 28],
        [21, 21, 21, ..., 28, 28, 28],
        [ 8,  8,  8, ..., 28, 28, 28],
        ...,
        [ 3,  3,  3, ..., 28, 28, 28],
        [27, 27, 27, ..., 28, 28, 28],
        [27, 27, 27, ..., 28, 28, 28]])>,
 'scrim_ids': <tf.Tensor: shape=(32, 256), dtype=int64, numpy=
 array([[19, 19, 19, ..., 99, 99, 99],
        [34, 34, 34, ..., 

In [31]:
custom_loss = CustomSparseCategoricalCrossentropy(from_logits=True, class_weights=weights_inv)

model_small.compile(optimizer=tf.keras.optimizers.Adam(),
                    loss=custom_loss,
                    metrics=[CustomSparseCategoricalAccuracy(),
                             CustomTopKAccuracy(k=3, name='custom_top_3_accuracy'),
                             CustomTopKAccuracy(k=5, name='custom_top_5_accuracy')])

history_small = model_small.fit(training_data, validation_data = testing_data, epochs=9, callbacks = [schedule])

pd.DataFrame(history_small.history).to_csv("models/modeling/history/training_history_model_small.csv", index = False, sep = ";")

model_small.save_weights("models/modeling/weights/QBGPT/model_small/QBGPT")

Epoch 1/9
 113/6433 [..............................] - ETA: 58:34 - loss: 3.3350 - custom_sparse_categorical_accuracy: 0.3415 - custom_top_3_accuracy: 0.4864 - custom_top_5_accuracy: 0.5427

KeyboardInterrupt: 

In [ ]:
model_small.summary()

Model: "qbgpt_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_2 (Encoder)         multiple                  1939584   
                                                                 
 dense_8 (Dense)             multiple                  1440156   
                                                                 
Total params: 3379740 (12.89 MB)
Trainable params: 3379484 (12.89 MB)
Non-trainable params: 256 (1.00 KB)
_________________________________________________________________


In [ ]:
custom_loss = CustomSparseCategoricalCrossentropy(from_logits=True, class_weights=weights_inv)

model_medium.compile(optimizer=tf.keras.optimizers.Adam(),
                     loss=custom_loss,
                     metrics=[CustomSparseCategoricalAccuracy(),
                              CustomTopKAccuracy(k=3, name='custom_top_3_accuracy'),
                              CustomTopKAccuracy(k=5, name='custom_top_5_accuracy')])

history_medium = model_medium.fit(training_data, validation_data = testing_data, epochs=9, callbacks = [schedule])

pd.DataFrame(history_medium.history).to_csv("training_history_model_medium.csv", index = False, sep = ";")

model_medium.save_weights("models/model_medium/QBGPT")

Epoch 1/9
6433/6433 [==============================] - 472s 70ms/step - loss: 1.4208 - custom_sparse_categorical_accuracy: 0.5094 - custom_top_3_accuracy: 0.7895 - custom_top_5_accuracy: 0.9008 - val_loss: 1.2922 - val_custom_sparse_categorical_accuracy: 0.5306 - val_custom_top_3_accuracy: 0.8140 - val_custom_top_5_accuracy: 0.9213 - lr: 0.0030
Epoch 2/9
6433/6433 [==============================] - 316s 47ms/step - loss: 1.2714 - custom_sparse_categorical_accuracy: 0.5351 - custom_top_3_accuracy: 0.8196 - custom_top_5_accuracy: 0.9256 - val_loss: 1.2455 - val_custom_sparse_categorical_accuracy: 0.5413 - val_custom_top_3_accuracy: 0.8259 - val_custom_top_5_accuracy: 0.9298 - lr: 0.0020
Epoch 3/9
6433/6433 [==============================] - 310s 46ms/step - loss: 1.2320 - custom_sparse_categorical_accuracy: 0.5456 - custom_top_3_accuracy: 0.8300 - custom_top_5_accuracy: 0.9324 - val_loss: 1.2201 - val_custom_sparse_categorical_accuracy: 0.5490 - val_custom_top_3_accuracy: 0.8327 - val_cu

In [ ]:
model_medium.summary()

Model: "qbgpt_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_4 (Encoder)         multiple                  4337920   
                                                                 
 dense_20 (Dense)            multiple                  2869148   
                                                                 
Total params: 7207068 (27.49 MB)
Trainable params: 7206556 (27.49 MB)
Non-trainable params: 512 (2.00 KB)
_________________________________________________________________


In [20]:
custom_loss = CustomSparseCategoricalCrossentropy(from_logits=True, class_weights=weights_inv)

model_large.compile(optimizer=tf.keras.optimizers.Adam(),
                    loss=custom_loss,
                    metrics=[CustomSparseCategoricalAccuracy(),
                             CustomTopKAccuracy(k=3, name='custom_top_3_accuracy'),
                             CustomTopKAccuracy(k=5, name='custom_top_5_accuracy')])

history_large = model_large.fit(training_data, validation_data = testing_data, epochs=9, callbacks = [schedule])

pd.DataFrame(history_large.history).to_csv("training_history_model_large.csv", index = False, sep = ";")

model_large.save_weights("models/model_large/QBGPT")

Epoch 1/9
6433/6433 [==============================] - 555s 82ms/step - loss: 1.4149 - custom_sparse_categorical_accuracy: 0.5107 - custom_top_3_accuracy: 0.7911 - custom_top_5_accuracy: 0.9023 - val_loss: 1.2823 - val_custom_sparse_categorical_accuracy: 0.5320 - val_custom_top_3_accuracy: 0.8168 - val_custom_top_5_accuracy: 0.9239 - lr: 0.0030
Epoch 2/9
6433/6433 [==============================] - 344s 51ms/step - loss: 1.2584 - custom_sparse_categorical_accuracy: 0.5381 - custom_top_3_accuracy: 0.8231 - custom_top_5_accuracy: 0.9282 - val_loss: 1.2386 - val_custom_sparse_categorical_accuracy: 0.5431 - val_custom_top_3_accuracy: 0.8283 - val_custom_top_5_accuracy: 0.9313 - lr: 0.0020
Epoch 3/9
6433/6433 [==============================] - 329s 49ms/step - loss: 1.2175 - custom_sparse_categorical_accuracy: 0.5491 - custom_top_3_accuracy: 0.8339 - custom_top_5_accuracy: 0.9353 - val_loss: 1.2119 - val_custom_sparse_categorical_accuracy: 0.5505 - val_custom_top_3_accuracy: 0.8351 - val_cu

In [21]:
model_large.summary()

Model: "qbgpt"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  10510848  
                                                                 
 dense_2 (Dense)             multiple                  5727132   
                                                                 
Total params: 16237980 (61.94 MB)
Trainable params: 16236956 (61.94 MB)
Non-trainable params: 1024 (4.00 KB)
_________________________________________________________________


In [33]:
custom_loss = CustomSparseCategoricalCrossentropy(from_logits=True, class_weights=weights_inv)

large_model.compile(optimizer=tf.keras.optimizers.Adam(),
                    loss=custom_loss,
                    metrics=[CustomSparseCategoricalAccuracy(),
                             CustomTopKAccuracy(k=3, name='custom_top_3_accuracy'),
                             CustomTopKAccuracy(k=5, name='custom_top_5_accuracy')])

history_large = large_model.fit(training_data, validation_data = testing_data, epochs=9, callbacks = [schedule])

pd.DataFrame(history_large.history).to_csv("training_history_large_model.csv", index = False, sep = ";")

large_model.save_weights("models/large_model/QBGPT")

Epoch 1/9
6433/6433 [==============================] - 533s 80ms/step - loss: 1.4355 - custom_sparse_categorical_accuracy: 0.5069 - custom_top_3_accuracy: 0.7862 - custom_top_5_accuracy: 0.8982 - val_loss: 1.2904 - val_custom_sparse_categorical_accuracy: 0.5297 - val_custom_top_3_accuracy: 0.8129 - val_custom_top_5_accuracy: 0.9211 - lr: 0.0030
Epoch 2/9
6433/6433 [==============================] - 336s 50ms/step - loss: 1.2792 - custom_sparse_categorical_accuracy: 0.5332 - custom_top_3_accuracy: 0.8172 - custom_top_5_accuracy: 0.9238 - val_loss: 1.2522 - val_custom_sparse_categorical_accuracy: 0.5398 - val_custom_top_3_accuracy: 0.8235 - val_custom_top_5_accuracy: 0.9283 - lr: 0.0020
Epoch 3/9
6433/6433 [==============================] - 329s 49ms/step - loss: 1.2426 - custom_sparse_categorical_accuracy: 0.5432 - custom_top_3_accuracy: 0.8268 - custom_top_5_accuracy: 0.9302 - val_loss: 1.2241 - val_custom_sparse_categorical_accuracy: 0.5482 - val_custom_top_3_accuracy: 0.8314 - val_cu

In [34]:
large_model.summary()

Model: "large_qbgpt_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_l_1 (EncoderL)      multiple                  5127936   
                                                                 
 dense_26 (Dense)            multiple                  2869148   
                                                                 
Total params: 7997084 (30.51 MB)
Trainable params: 7996060 (30.50 MB)
Non-trainable params: 1024 (4.00 KB)
_________________________________________________________________
